In [1]:
import cv2
from ultralytics import YOLO
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel
from PyQt5.QtCore import QTimer
from PyQt5.QtGui import QImage, QPixmap
from adbutils import adb
import scrcpy.core as scrcpy
import sys

In [2]:
phones = ['Pixel 3 XL', 'Pixel 7 Pro']
adb.connect("127.0.0.1:5037")
client = scrcpy.Client(device=adb.device_list()[0])
client.start(threaded=True)
print(client.device_name)
phone_t = phones.index(client.device_name)

Pixel 3 XL


In [3]:
# Load the YOLOv8 model
model = YOLO("yolov8n.pt")

update_timer = 0

def yolo_inference(image):
    # Perform inference
    results = model(image)

    # Debug: print out the type and content of results
    print(type(results))
    print(results)
    print(type(results[0].boxes))
    print(results[0].boxes)
    # Draw results on the image
    for result in results.pred[0]:  # assuming you have one image, so results.pred[0]
        result = result.tolist()
        x1, y1, x2, y2, conf, cls = result
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(image, f'{int(cls)} {conf:.2f}', (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

    return image





def update_ui():
    screen = client.last_frame

    if screen is not None:
        # Perform YOLO inference and get results
        screen = yolo_inference(screen)
        
        scale_percent = 20
        width = int(screen.shape[1] * scale_percent / 100)
        height = int(screen.shape[0] * scale_percent / 100)
        dim = (width, height)
        resized_image = cv2.resize(screen, dim, interpolation=cv2.INTER_AREA)
        resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)

        height, width, channel = resized_image.shape
        bytes_per_line = channel * width
        qimage = QImage(resized_image.data, width, height, bytes_per_line, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qimage)
        screenshot_label.setPixmap(pixmap)

    QTimer.singleShot(update_timer, update_ui)


app = QApplication(sys.argv)
window = QWidget()
window.setWindowTitle("Pokémon Information Display")
layout = QVBoxLayout()


screenshot_label = QLabel()
layout.addWidget(screenshot_label)

window.setLayout(layout)
window.show()

# Start the update loop
QTimer.singleShot(update_timer, update_ui)

app.exec_()


QSocketNotifier: Can only be used with threads started with QThread

0: 640x320 (no detections), 35.1ms
Speed: 3.4ms preprocess, 35.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)


<class 'list'>
[ultralytics.yolo.engine.results.Results object with attributes:

boxes: ultralytics.yolo.engine.results.Boxes object
keypoints: None
keys: ['boxes']
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56:

AttributeError: 'list' object has no attribute 'pred'